### Logic

This covers topics from chapters 7, 8, and 9 in AIMA

#### Contents 
- Logical Sentences 
    - Expr
    - PropKB
    - Knowledge-Based Agents 
    - Inference in Propositional Knowledge Bases 
        - Truth Table Enumeration 
        - Proof by Resolution
        - Forward and Backward Chaining
        - DPLL
        - WalkSAT 
        - SATPlan
    - FolKB
    - Inference in First Order Knowledge Base 
        - Unification
        - Forward Chaining
        - Backward Chaining

In [1]:
# import libraries 
from utils import * 
from logic import * 
from notebook import psource

#### Logical Sentences

The Expr class is designed to represent any kind of mathematical expression.

In [2]:
# define a symbol
Symbol('x')

# define multiple symbols 
(x, y, P, Q, f) = symbols('x, y, P, Q, f')

# combine expressions with infix and prefix operators
# This works because the Expr class overloads the & operator
P & ~Q

(P & ~Q)

In [3]:
# look at fields for expressions 
sentence = P & ~Q

# operator
print("operator: ", sentence.op)

# arguments 
print("arguments: ", sentence.args)

# Symbol operator 
print("P operator: ", P.op)

# Symbol arguments
print("P arguments: ", P.args)

# define a bigger expression 
Pxy = P(x, y)

# bigger expression operator
print("Pxy operator: ", Pxy.op)

# bigger expression arguments
print("Pxy arguments: ", Pxy.args)

operator:  &
arguments:  (P, ~Q)
P operator:  P
P arguments:  ()
Pxy operator:  P
Pxy arguments:  (x, y)


We should note that the Expr class does not define the logic of Propositional logic sentences; it just gives a way to represent expressions. We can think of it as an abstract syntax tree.

In [10]:
# deeply nested Expr
3 * f(x, y) + P(y) / 2 + 1

(((3 * f(x, y)) + (P(y) / 2)) + 1)

#### Operators for Constructing Logical Sentences 

In [4]:
# define a sentence with an implication arrow 
~(P & Q) |'==>'| (~P | ~Q)

(~(P & Q) ==> (~P | ~Q))

#### Propositional Knowledge Bases: PropKB

The class PropKB can be used to represent a knowledge base of propositional logic sentences. 

The class PropKB: 
    - __init__(self, sentence = None): The constructor __init__ creates a single field clauses which will be a list of all the sentences of the knowledge base. Note that each one of these sentences will be a 'clause' i.e. a sentence which is made up of only literals and ors. 
    
    - tell(self, sentence): When you want to add a sentence to the KB, you use the tell method. This method takes a sentence, converts it to its CNF, extracts all the clauses, and adds all these clauses to the clauses field. So, you need not worry about telling only clauses to the knowledge base. You can tell the knowledge base a sentence in any form that you wish; converting it to CNF and adding the resulting clauses will be handled by the tell method. 
    
    - ask_generator(self, query): The ask_generator function is used by the ask function. It calls the tt_entails function, which in turn returns True if the knowledge base entails query and false otherwise. The ask_generator itself returns an empty dict {} if the knowledge base entails query and None otherwise. This is done to maintain consistency with the way things are in First-Ordeer Logic, where an ask_generator function is supposed to return all the substritutions that make the query true. 
    
    - retract(self, sentence): This function removes all the clauses of the sentence given, from th eknowledge base. Like the tell function, we don't need to pass clauses to remove them from the knowledge base; any sentence will do. This function will take care of convertaing the sentence to clauses. 

In [6]:
# Create a PropKB for the wumpus world
wumpus_kb = PropKB()

# Px,y is true if there is a pit in [x, y]
# Bx,y is true if the agent senses breeze in [x, y]
P11, P12, P21, P22, P31, B11, B21 = expr('P11, P12, P21, P22, P31, B11, B21')

# enter sentences 

# there is no pit in [1,1]
wumpus_kb.tell(~P11)

# A square is breezy iff there is a pit in the neighboring square
wumpus_kb.tell(B11 | '<=>' | ((P12 | P21)))
wumpus_kb.tell(B21 | '<=>' | ((P11 | P22 | P31)))

# add breeze percepts 
wumpus_kb.tell(~B11)
wumpus_kb.tell(B21)

# check clauses stored in the KB
wumpus_kb.clauses

[~P11,
 (~P12 | B11),
 (~P21 | B11),
 (P12 | P21 | ~B11),
 (~P11 | B21),
 (~P22 | B21),
 (~P31 | B21),
 (P11 | P22 | P31 | ~B21),
 ~B11,
 B21]

From the above printout we see the storage of all the inputs we placed into our knowledge base. We can see that PropKB did some conversions to equivalences, such as B11 <-> (P12 v P21)

#### Knowledge Based Agents 

A knowledge based agent is a simple generic agent that maintains and handles a knowledge base. The knowledge base may initially contain some background knowledge. The purpose of a KB agent is to provide a level of abstraction over knowledge-base manipulation and is to be used as a base class for agents that work on a knowledge base. 

Given a percept, the KB agent adds the percept to its knowledge base, then asks the knowledge base for the best action, and tells the knowledge base that it has taken that action. Our implementation of KB-Agent is encapsulated in a class KB_AgentProgram which inherits from the KB class.


In [7]:
psource(KB_AgentProgram)

make_percept_sentence makes first order logic sentences about percepts we want our agent to recieve

make_action_query asks the underlying KB about the action that should be taken

make_action_sentence tells the underlying KB about the action it has just taken

### Inference in Propositional Knowledge Bases

In this section we will look at two algorithms to check if a sentence is entailed by the KB. Our goal is to decide whether KB models alpha for some sentence alpha

#### Truth Table Enumeration

It is a model checking approach which enumerates all possible models in which the KB is true and checks if alpha is also true in these models. We list the n symbols in the KB and enumerate the 2^n models in a depth first manner and then we check the truth of KB and alpha. 

The algorithm basically computes every line of the truth table KB -> alpha and checks if it is true everywhere. If symbols are defined, the routine recursively constructs every combination of truth tables for the symbols and then checks whether model is consistent with KB. The given models correspond to the lines in the truth table, which have a true in the KB column, and for these lines it checks whether the query evaluates to true. 

In [8]:
psource(tt_check_all)

In [9]:
psource(tt_entails)

In [14]:
# try out 
print("Does P&Q -> Q?", tt_entails(P&Q, Q))
print("Does P or Q -> Q?", tt_entails(P|Q, Q))
print("Does P or Q -> P?", tt_entails(P|Q, P))

# define new symbols for a larger sentence
(A, B, C, D, E, F, G) = symbols('A, B, C, D, E, F, G')
print("Does A & (B v C) & D & E & not (F v G) entail A & D & E & not F & not G?", tt_entails(A & (B | C) & D & E & ~(F | G), A & D & E & ~F & ~G))

Does P&Q -> Q? True
Does P or Q -> Q? False
Does P or Q -> P? False
Does A & (B v C) & D & E & not (F v G) entail A & D & E & not F & not G? True


In [15]:
# try it out in the wumpus world 
print("Is the wumpus (not in | in) square P11?", wumpus_kb.ask_if_true(~P11), wumpus_kb.ask_if_true(P11))

Is the wumpus (not in | in) square P11? True False


#### Proof by Resolution

This technique corresponds to a proof by contradictyion. We assume alpha to be false and show that this leads to a contradiction with known axions in KB. We obtain a contradiction by making valid inferences using inference rules. In this technique we use a single inference rule called resolution which states (l1 v ... v lk) ^ (m1 v ... v mn) ^ (li <-> not mj) => l1 v ... v li-1 v li+1 v ... v lk v m1 v ... v mj-1 v mj+1 v ... v mn

Applying resolution yields us a clause which we add to the  KB. We keep doing this until 
    - There are no new clauses that can be added, in which case KB does not entail alpha
    - Two clauses resolve to yield the empty clause, in which case KB entails alpha
    
There is one catch: The algorithm that implements proof by resolution cannot handle complex sentences. Implications and bi-implications have to be simplified into simpler clauses. Since we know that every sentence of a propositional logic is logically equivalent to a conjunction of clauses, we can use this fact and simplify our input sentence into its conjunctive normal form. 

In [16]:
# look at sources 

# convert to conjunctive normal form
psource(to_cnf)

# eliminiate implications comverts biimplications and implications to their logical equivalents
psource(eliminate_implications)

# move_not_inwards removes negations from compound statements and moves them inwards using DeMorgans Laws
psource(move_not_inwards)

# distributuve_and_over_or distributes disjunctions over conjunctions 
psource(distribute_and_over_or)

In [25]:
# convert some sentences 
A, B, C, D = expr('A, B, C, D')
print("Convert A <=> B to CNF: ", to_cnf(A |'<=>'| B))
print("Convert A <=> (B&C) to CNF: ", to_cnf(A |'<=>'| (B&C)))
print("Convert A <=> B v (C & D) to CNF: ", to_cnf(A |'<=>'| (B | (C & D))))
print("Convert A <=> not B implies C v not D to CNF: ", to_cnf((A |'<=>'| B) |'==>'| (C | ~D)))

Convert A <=> B to CNF:  ((A | ~B) & (B | ~A))
Convert A <=> (B&C) to CNF:  ((A | ~B | ~C) & (B | ~A) & (C | ~A))
Convert A <=> B v (C & D) to CNF:  ((~B | A) & (~C | ~D | A) & (C | B | ~A) & (D | B | ~A))
Convert A <=> not B implies C v not D to CNF:  ((~B | ~A | C | ~D) & (A | ~A | C | ~D) & (~B | B | C | ~D) & (A | B | C | ~D))


#### Forward and Backward Chaining

This algorithm is also looking to check if a sentence is entailed by the KB, but the difference is that our goal now is to determine if a knowledge base of definite clauses entails a single propositional symbol q - the query. The catch is that the knowledge base only contains horn clauses. 

Horn Clauses can be defined as a disjunction of literals with at most one positive literal. 
A Horn Clause with exactly one positive literal is called a definite clause. A Horn Clause might look like ~a v ~b v ... v ~y v z. This coincidentally is also a definite clause. Using DeMorgan's laws the above can be simplifed to a^b^...^y => z. 

There are some aspects of Horn Clauses that make algorithmic inference or resolution easier: 

- Definite clauses can be written as implications 
- Forward chaining and backward chaining can be used for inference from Horn Clauses 
- Deciding entailment with Horn Clauses is linear in size of the knowledge base


In [26]:
# PropDefiniteKD returns a list of clauses in KB that have a given symbol p in their premise
psource(PropDefiniteKB.clauses_with_premise)

# pl_fc_entails implements forward chaining to see if a knowledge base entails a query q
psource(pl_fc_entails)

In [27]:
# define our knowledge base
clauses = ['(B & F)==>E', 
           '(A & E & F)==>G', 
           '(B & C)==>F', 
           '(A & B)==>D', 
           '(E & F)==>H', 
           '(H & I)==>J',
           'A', 
           'B', 
           'C']
 
# instantiate KB
definite_clauses_KB = PropDefiniteKB() 

# add clauses to KB
for clause in clauses: 
    definite_clauses_KB.tell(expr(clause))

In [29]:
# check to see if our KB entails the following queries 
print("Does our KB entail G?", pl_fc_entails(definite_clauses_KB, expr('G')))
print("Does our KB entail H?", pl_fc_entails(definite_clauses_KB, expr('H')))
print("Does our KB entail I?", pl_fc_entails(definite_clauses_KB, expr('I')))
print("Does our KB entail J?", pl_fc_entails(definite_clauses_KB, expr('J')))

Does our KB entail G? True
Does our KB entail H? True
Does our KB entail I? False
Does our KB entail J? False


#### Effective Propositional Model Checking 

The previous segments elucidate the algorithmic procedure for model checking. In this segment, we look at ways of making them computationally efficient. 

The problem we are trying to solve is called the Propositional Satisfiability Problem (SAT Problem). If there exists a model that satisfies a given Boolean formula, the formula is called satisfiable. The SAT problem was the first problem to be proven NP-complete. The main characteristics of an NP-complete problem are: 
    - Given a solution to such a problem, it is easy to verify if the solution solves the problem
    - The time required to solve the problem using any known algorithm increases exponentially with respect to the size of the problem 
    
Due to these properties, heuristic and approximational methods are often applied to find solutions to these problems. It is important to solve these problems because many combinatorial problems in computer science can be conveniently reduced to checking the satisfiablity of a propositional sentence under some constraints.

#### DPLL (David Putnam Logeman Loveland) Algorithm

This algorithm is similar to backtracking-search. It recursively enumerates possible models in a depth-first fashion with the following improvements over algorithms like tt_entails: 

    - Early Termination 
    - Pure Symbol Heuristic
        - A symbol with the same sign in all clauses is called a pure symbol
    - Unit Clause Heuristic
        - Clauses with just one literal and clauses with all but one false literal are called unit clauses. If a clause is a unit clause, it can only be satisfied by assigning the necessary value to make the last literal true.


In [30]:
# dpll recursively checks satisfiability in propositional logic 
psource(dpll)

# dpll satisfiable is a helper function that converts the input clauses to conjunctive normal form and calls the dpll function with the correct parameters 
psource(dpll_satisfiable)

In [36]:
# examples 
A, B, C, D = expr('A, B, C, D')
print("Is there a model that satisfies A & B & ~C & D ? ", dpll_satisfiable(A & B & ~C & D))
print("Is there a model that satisfies (A & B) | (C & ~A) | (B & ~D) ? ", dpll_satisfiable((A & B) | (C & ~A) | (B & ~D)))
print("Is there a model that satisfies A |'<=>'| B ? ", dpll_satisfiable(A |'<=>'| B))
print("Is there a model that satisfies (A |'<=>'| B) |'==>'| (C & ~A) ? ", dpll_satisfiable((A |'<=>'| B) |'==>'| (C & ~A)))
print("Is there a model that satisfies (A | (B & C)) |'<=>'| ((A | B) & (A | C)) ? ", dpll_satisfiable((A | (B & C)) |'<=>'| ((A | B) & (A | C))))

Is there a model that satisfies A & B & ~C & D ?  {A: True, B: True, D: True, C: False}
Is there a model that satisfies (A & B) | (C & ~A) | (B & ~D) ?  {B: True, D: False}
Is there a model that satisfies A |'<=>'| B ?  {A: True, B: True}
Is there a model that satisfies (A |'<=>'| B) |'==>'| (C & ~A) ?  {C: True, A: True, B: False}
Is there a model that satisfies (A | (B & C)) |'<=>'| ((A | B) & (A | C)) ?  {A: True}


#### WalkSAT Algorithm

This algorithm is similar to the hill climbing algorithms. On every iteration, the algorithm picks an unsatisfied clause and flips a symbol in the clause. This is similar to finding a neighboring state in the hill_climbing algorithm. 

The symbol to be flipped is decided by an evaluation function that counts the number of unsatisfied clauses. Sometimes symbols are also flipped randomly to avoid local optima. A subtle balance between greediness and randomness is required. Alternatively, some versions of the algorithm restart with a completely new random assignment if no solution has been found for too long as a way of getting out of local minima of numbers of unsatisfied clauses.

In [37]:
'''
This function takes 3 arguments: 
    - The clauses we want to satisfy
    - The probability p of randomly changing a symbol
    - The maximum number of flips (max_flips) the algorithm will run for. If the clauses are still unsatisfied, the algorithm returns none to denote failure
'''

psource(WalkSAT)

In [39]:
# define domain
A, B, C, D = expr('A, B, C, D')

print("Is A, B, ~C, D satisfiable? ", WalkSAT([A, B, ~C, D], 0.5, 100))
print("Is A & B, A & C satisfiable? ", WalkSAT([A & B, A & C], 0.5, 100))
print("Is A & B, C & D, C & B satisfiable? ", WalkSAT([A & B, C & D, C & B], 0.5, 100))
print("Is A & B, C | D, ~(D | B) satisfiable? ", WalkSAT([A & B, C | D, ~(D | B)], 0.5, 1000))


Is A, B, ~C, D satisfiable?  {A: True, B: True, D: True, C: False}
Is A & B, A & C satisfiable?  {A: True, B: True, C: True}
Is A & B, C & D, C & B satisfiable?  {A: True, B: True, D: True, C: True}
Is A & B, C | D, ~(D | B) satisfiable?  None


In [40]:
# write a helper function to allow WalkSAT to take complete sentences as input.
# input -> convert to CNF -> call WalkSAT
def WalkSAT_CNF(sentence, p = 0.5, max_flips = 10000): 
    return WalkSAT(conjuncts(to_cnf(sentence)), 0, max_flips)

In [43]:
print("Is (A & B) | (C & ~A) | (B & ~D) satisfiable ? ", WalkSAT_CNF((A & B) | (C & ~A) | (B & ~D), 0.5, 1000))

Is (A & B) | (C & ~A) | (B & ~D) satisfiable ?  {A: False, B: True, D: False, C: True}


In [48]:
# compare the runtime of WalkSAT and DPLL

# define sentences 
sentence_1 = A |'<=>'| B
sentence_2 = (A & B) | (C & ~A) | (B & ~D)
sentence_3 = (A | (B & C)) |'<=>'| ((A | B) & (A | C))

# timer
print("DPLL Times : \n")
%timeit dpll_satisfiable(sentence_1)
%timeit dpll_satisfiable(sentence_2)
%timeit dpll_satisfiable(sentence_3)
print("\n")

print("WalkSAT Times : \n")
%timeit WalkSAT_CNF(sentence_1)
%timeit WalkSAT_CNF(sentence_2)
%timeit WalkSAT_CNF(sentence_3)
print("\n")

DPLL Times : 

252 µs ± 5.55 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.03 ms ± 27.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.57 ms ± 8.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


WalkSAT Times : 

214 µs ± 3.46 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
896 µs ± 7.69 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.22 ms ± 9.08 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)




On average, for solvable cases, WalkSAT is quite a bit faster than DPLL because WalkSAT can reduce the search space significantly. Results can be different for sentences with more symbols.


#### SATPlan



In this section we show how to make plans by logical inference. The basic idea is simple: 
    1. Construct a sentence that includes: 
        A. A collection of assertions about the initial state
        B. The successor state axioms for all the possible actions at each time up to some maximum t
        c. The assertion that the goal is achieved at time t
    2. Present the whole sentence to a SAT solver
    3. Assuming a model is found, extract from the model those variables that represent actions and are assigned true. Together they represent a plan to achieve the goals. 
     

In [49]:
psource(SAT_plan)

In [52]:
# examples of SATPlan
print("Transition 1 \n")
transition = {'A': {'Left': 'A', 'Right': 'B'},
            'B': {'Left': 'A', 'Right': 'C'},
            'C': {'Left': 'B', 'Right': 'C'}}

print(SAT_plan('A', transition, 'C', 2)) 
print(SAT_plan('A', transition, 'B', 3))
print(SAT_plan('C', transition, 'A', 3))
print("\n")

print("Transition 2 \n")
transition = {(0, 0): {'Right': (0, 1), 'Down': (1, 0)},
            (0, 1): {'Left': (1, 0), 'Down': (1, 1)},
            (1, 0): {'Right': (1, 0), 'Up': (1, 0), 'Left': (1, 0), 'Down': (1, 0)},
            (1, 1): {'Left': (1, 0), 'Up': (0, 1)}}

print(SAT_plan((0, 0), transition, (1, 1), 4))

Transition 1 

None
['Right']
['Left', 'Left']


Transition 2 

['Right', 'Down']


### First Order Logic Knowledge Bases: FolKB 

The class FolKB can be used to represent a knowledge base of First-Order logic sentences. You can initialize it and use it in the same way as you would for PropKB except that the clauses are first-order definite clauses. We will see how to write such clauses to create a database and query them in the following sections.

#### Criminal KB

In this section we create a FolKB based on the following paragraph: 

The law says that it is a crime for an American to sell weapons to hostile nations. The country Nono, an enemy of America, has some missiles, and all of its missiles were sold to it by Colonel West, who is American.

The first step is to extract the facts and convert them into first order definite clauses.

In [53]:
# create list to store clauses 
clauses = []

# append clauses 
clauses.append(expr("(American(x) & Weapon(y) & Sells(x, y, z) & Hostile(z)) ==> Criminal(x)"))
clauses.append(expr("Enemy(Nono, America)"))
clauses.append(expr("Owns(Nono, M1)"))
clauses.append(expr("Missile(M1)"))
clauses.append(expr("(Missile(x) & Owns(Nono, x)) ==> Sells(West, x, Nono)"))
clauses.append(expr("American(West)"))
clauses.append(expr("Missile(x) ==> Weapon(x)"))
clauses.append(expr("Enemy(x, America) ==> Hostile(x)"))

In [55]:
# instantiate first order knowledge base 
crime_kb = FolKB(clauses)

# helper function subst substitues variables with given variables in first order logic statements
psource(subst)

# subst example
subst({x: expr('Nono'), y: expr('M1')}, expr('Owns(x, y)'))

Owns(Nono, M1)

### Inference in First Order Logic 

In this section we look at a forward chaining and backward chaining algorithm for FolKB. Both aforementioned algorithms rely on a process called unification, a key component in all first order inference algorithms.

#### Unification

We sometimes require finding substitutions that make different logical expressions look identical. This process, called unification, is done with the unify algorithm. It takes as input two sentences and returns a unifier for them if one exists. A unified is a dictionary which stores the substitutions requires to make the two sentences identical. It doees so by recursively unifying the components of a sentence, where the unification of a variable symbol var with a constant symbol const is the mapping {var: Const}

In [60]:
# unify examples 
print("set x to 3 : ", unify(expr('x'), 3))
print("set A(x) to A(b)", unify(expr('A(x)'), expr('A(B)')))
print("set Cat(x) & Dog(Dobby) to Cat(Bella) & Dog(y)", unify(expr('Cat(x) & Dog(Dobby)'), expr('Cat(Bella) & Dog(y)')))

# in cases where there is no possible solution that unifies the two sentences the function returns None
print(unify(expr('Cat(x)'), expr('Dog(Dobby)')))

# we also must check that we do not unintentionally use the same variable name.
print(unify(expr('Cat(x) & Dog(Dobby)'), expr('Cat(Bella) & Dog(x)')))

set x to 3 :  {x: 3}
set A(x) to A(b) {x: B}
set Cat(x) & Dog(Dobby) to Cat(Bella) & Dog(y) {x: Bella, y: Dobby}
None
None


#### Forward Chaining Algorithm

We consider the simple forward chaining algorithm. We look at each rule in the knowledge base and see if the premises can be satisfied. This is done by finding a substitution which unifies each of the premises with a clause in the KB. This inferencing process is repeated until either the query can be answered or until no new sentences can be added. We test if the newly added clause unifies with the query in which case the substitution yieled by unify is an answer to the query. If we run out of sentences to infer, then the query was a failure. 

In [61]:
# fol_fc_ask is a generator which yields all substitutions which validate the query
psource(fol_fc_ask)

In [63]:
# find all the hostile nations
answer = fol_fc_ask(crime_kb, expr('Hostile(x)'))
print(list(answer))

[{x: Nono}]


In [64]:
crime_kb.tell(expr('Enemy(JaJa, America)'))
answer = fol_fc_ask(crime_kb, expr('Hostile(x)'))
print(list(answer))

[{x: Nono}, {x: JaJa}]


#### Backward Chaining Algorithm

This algorithm works backward from the goal, chaining through rules to find known facts that support the proof. Suppose goal is the query we want to find the substitution for. We find rules of the form lhs => goal in the KB to try to prove lhs. There may be multiple classes in the KB which give multiple lhs. It is sufficient to prove only one of these. To prove a lhs all conjuncts in the lhs of the clause must be proven. This makes it similar to AND/OR search. 

The OR part of the algorithm comes from our choice to select any clause of the form lhs => goal. Looking at all rules lhs whose rhs unifies with the goal, we yield a substitution which proves all the conjuncts in the lhs. We use parse_definite_clause to attain lhs and rhs from a clause of the form lhs => rhs. 

In [66]:
# or
psource(fol_bc_or)

# and
psource(fol_bc_and)

In [67]:
# rebuild KB because running fol_fc_ask would add new facts to the KB 
crime_kb = FolKB(clauses)
crime_kb.ask(expr('Hostile(x)'))

{v_5: x, x: Nono}